In [ ]:
import ee
import geemap

In [ ]:
!gcloud

In [7]:
ee.Initialize()

In [34]:
projectId = 'qwiklabs-gcp-01-d5556e0a3f77';

# load in the results from DINEOF process
dineofResults = ee.ImageCollection('projects/' + projectId + '/assets/dineof_cloud_run_demo');

# get the region from the DINEOF result
region = dineofResults.first().geometry();

# get the dates that DINEOF ran for
dates = dineofResults.aggregate_array('system:time_start');

# load in the original data and filter to the dates that DINEOF ran for
original = ee.ImageCollection('NASA/OCEANDATA/MODIS-Aqua/L3SMI').select('sst').map(
      lambda img: img.set('date',ee.Date(img.date().format('YYYY-MM-dd')).millis())
    ).filter(ee.Filter.inList('date', dates))

# get the land mask from MODIS for visualization
landMask = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24').select('water_mask').unmask(1).Not().selfMask()

# define visualization parameters for the sst data
sstVis = {
  "min":15,
  "max": 35,
  "palette": ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
  "bands":"sst"
}

# Create a  geemap Map instance
m = geemap.Map()

# add the layers to the map
m.addLayer(original.first().clip(region),sstVis, 'Original');
m.addLayer(dineofResults.first(), sstVis, 'DINEOF');
m.addLayer(landMask, {"min":0, "max":1, "palette":'silver'}, 'Land mask', False);
m.centerObject(dineofResults)

m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [42]:
from IPython.display import Image

# create parameters to create a gif
videoParams = {
  "region": region,
  "crs": 'epsg:4326',
  "framesPerSecond": 2,
  "dimensions": 300,
  # "bands": ["red", "green", "blue"]
}

# function to visualize the results and add the land mask
def gifPrep(img):
  return img.visualize(**sstVis).blend(
      landMask.visualize(min=0,
                         max=1,
                         palette="silver",
                         bands=["water_mask"])
  )

# create the gifs
originalGif = original.map(gifPrep)
dineofGif = dineofResults.map(gifPrep)

# print the gif result url
Image(url=originalGif.getVideoThumbURL(params=videoParams))

In [43]:
Image(url=dineofGif.getVideoThumbURL(params=videoParams))

<div class="markdown-google-sans">

## **Getting started**
</div>

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.

For example, the code cell below uses **numpy** to generate some random data, and uses **matplotlib** to visualize it. To edit the code, just click the cell and start editing.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

ys = 200 + np.random.randn(100)
x = [x for x in range(len(ys))]

plt.plot(x, ys, '-')
plt.fill_between(x, ys, 195, where=(ys > 195), facecolor='g', alpha=0.6)

plt.title("Sample Visualization")
plt.show()